# Building a Chatbot
## Part III: Building a chatbot UI

### CORE Studio, Thornton Tomasetti

#### Instructor: [Seyedomid Sajedi](https://www.linkedin.com/in/seyedomid-sajedi-263b703a)
Part I and II of exercise 4 taught us the mechanics of instructing the chatbot to answer user queries based on a custom databse that we define.

An important part of any machine learning project is how you share it with other people. We spent all our time executing code snippets in this workshop. For our final notebook, we will look at how [Streamlit](https://streamlit.io/) can help us build a good looking user interface for our chatbot. As expected, we need to bring the modules that we created in parts I and II and organize them.


##Libraries

In [1]:
%%capture
!pip install rank_bm25 pypdf2 tiktoken openai

In [3]:
# https://spacy.io/
import spacy
# !pip install --upgrade spacy # might be needed if the default spacy in colab is not working
import requests
from io import BytesIO
from PyPDF2 import PdfReader
from tqdm import tqdm
import tiktoken
from rank_bm25 import BM25Okapi
import matplotlib.pyplot as plt
import numpy as np
import pprint
import pickle

In [4]:
nlp_model = spacy.load("en_core_web_sm")

In [5]:
def spacy_tokenizer(text,nlp):
    doc = nlp(text)
    tokens = []
    for token in doc:
        # Check if the token is not punctuation and not a stop word, or if it's a stop word in all caps
        if (not token.is_punct) :
            # check if token letters are all cap
            all_cap_cond = all(c.isupper() for c in token.text)

            if not token.is_stop or ((token.is_stop) and (token.pos_ == 'PROPN' or all_cap_cond)):
                tokens.append(token.lemma_.lower())
    return tokens
# We used this function for topic modeling
def get_pdf_as_memory_stream(url):
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for HTTP errors

    # Convert the response content into a BytesIO stream
    return BytesIO(response.content)


In [6]:
# Ref: https://www.nyc.gov/site/buildings/codes/2022-construction-codes.page#bldgs
# make sure to click on the Local Law 77 from each chapter to get the pdf file.
pdf_url = r"https://www.nyc.gov/assets/buildings/local_laws/ll77of2023.pdf"
pdf_file = get_pdf_as_memory_stream(pdf_url)
reader = PdfReader(pdf_file)
# number of pages
n_pages = len(reader.pages)
print(f"Number of pages: {n_pages}")
# tokenize dataset
tokenized_dataset= []
text_dataset =[]
for page in tqdm(reader.pages,total=n_pages):
  text = page.extract_text()
  text_dataset.append(text)
  tokenized_page = spacy_tokenizer(text,nlp_model)
  tokenized_dataset.append(tokenized_page)
# save the dataset to a file to make it accecible for the chatbot
with open('dataset.pkl', 'wb') as f:
    pickle.dump({'text_dataset':text_dataset,
                 'tokenized_pages':tokenized_dataset},
                f)

Number of pages: 184


100%|██████████| 184/184 [00:30<00:00,  6.04it/s]


## Streamlit
Streamlit requires all your python script inside a py file. The first line is a magic command that will write all the instructions we have had before into a single `chatbot.py` file. Let's quickly review the code together.

In [8]:
%%writefile chatbot.py
# load libraries
import streamlit as st
import spacy
from rank_bm25 import BM25Okapi
import matplotlib.pyplot as plt
import numpy as np
import streamlit as st
import pickle
import openai
import os

# """
#   ____  _               _
#  / ___|| |_ ___ _ __   / |
#  \___ \| __/ _ \ '_ \  | |
#   ___) | ||  __/ |_) | | |
#  |____/ \__\___| .__/  |_|
#                |_|
# We define functions that load processed data and models into cache using streamlit's caching mechanism.
# The cache command make the streamlit app faster and more responsive.
# """
@st.cache_resource
def init_bm25_vectorizer():
    """Loads the bm25 vectorizer and puts it into cache"""
    global data_dict
    return BM25Okapi(data_dict['tokenized_pages'])

@st.cache_resource
def load_spacy_model():
    """Loads spacy tokenizer and puts it into cache"""
    return spacy.load("en_core_web_sm")

@st.cache_data
def load_data():
    """Loads the saved and processed data from the pickle file and stores in into cache"""
    with open('dataset.pkl', 'rb') as f:
        data_dict = pickle.load(f)
    return data_dict
# """

#   ____  _               ____
#  / ___|| |_ ___ _ __   |___ \
#  \___ \| __/ _ \ '_ \    __) |
#   ___) | ||  __/ |_) |  / __/
#  |____/ \__\___| .__/  |_____|
#                |_|
# We are organizing the functions that we defined in earlier steps of example 4.
# """
# Utility functions
def spacy_tokenizer(text,nlp):
    doc = nlp(text)
    tokens = []
    for token in doc:
        # Check if the token is not punctuation and not a stop word, or if it's a stop word in all caps
        if (not token.is_punct) :
            # check if token letters are all cap
            all_cap_cond = all(c.isupper() for c in token.text)

            if not token.is_stop or ((token.is_stop) and (token.pos_ == 'PROPN' or all_cap_cond)):
                tokens.append(token.lemma_.lower())
    return tokens

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    """This function handles calls to the openai api and returns the response from the chatgpt model"""
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def doc_retrieval(query,pages,
                  top_k=3):
  #  we need to load bm25_vectorizer,nlp_model in other scripts
  query_tokens = spacy_tokenizer(query,nlp_model)
  bm25_scores = bm25_vectorizer.get_scores(query_tokens)
  top_page_indx= np.argsort(bm25_scores)[-top_k:][::-1]
  hits = [{'page_indx': idx, 'score': bm25_scores[idx]} for idx in top_page_indx]
  ref_docs = [pages[hit['page_indx']] for hit in hits]
  return hits,ref_docs

def augment_prompt(prompt):
    doc_indx, ref_docs = doc_retrieval(prompt,data_dict['text_dataset'])
    aug_prompt = f"***{prompt}***"
    for doc in ref_docs:
        aug_prompt+=f"```{doc}```"
    st.session_state.chat_hist.append({'role':'user', 'content':prompt,'ref_docs':ref_docs})
    return aug_prompt

# """
#   ____  _               _____
#  / ___|| |_ ___ _ __   |___ /
#  \___ \| __/ _ \ '_ \    |_ \
#   ___) | ||  __/ |_) |  ___) |
#  |____/ \__\___| .__/  |____/
#                |_|

# Loading and caching data and models, adding openai credentials and defining the system prompt.
# """
# Initialize the data and models
data_dict = load_data()
nlp_model = load_spacy_model()
bm25_vectorizer = init_bm25_vectorizer()
# get openai api key
with open("secret_workshop.txt", "r") as f: # this is not the most secure way of adding secret key. For large scale deployment talk to your IT
    secret = f.read()
openai.api_key = secret



system_prompt="""You are a helpful assistant named Gary, your task is to review a series of\
documents returned by a search system and answer the user's question only based on these documents.\
The first user query is delimited by triple asterisks\.
The reference documents in that message are delimited with triple backticks.\
A user might ask follow up questions.
"""

# """
#   ____  _               _  _
#  / ___|| |_ ___ _ __   | || |
#  \___ \| __/ _ \ '_ \  | || |_
#   ___) | ||  __/ |_) | |__   _|
#  |____/ \__\___| .__/     |_|
#                |_|

# Streamlit chat interface.
# """

# Building a front end with streamlit
# ref: https://docs.streamlit.io/knowledge-base/tutorials/build-conversational-apps
st.title("AEC Workshop chatbot")

if "openai_model" not in st.session_state:
    st.session_state["openai_model"] = "gpt-3.5-turbo"

if "messages" not in st.session_state:
    st.session_state.messages = []
    st.session_state.chat_hist = []

for message in st.session_state.chat_hist:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

if prompt := st.chat_input("What is up?"):
    if len(st.session_state.chat_hist)==0:
        llm_prompt = augment_prompt(prompt)
    else:
        llm_prompt = prompt

    st.session_state.messages.append({"role": "user", "content": llm_prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    with st.chat_message("assistant"):
        message_placeholder = st.empty()
        full_response = ""
        for response in openai.ChatCompletion.create(
            model=st.session_state["openai_model"],
            messages=[
                {"role": m["role"], "content": m["content"]}
                for m in st.session_state.messages
            ],
            stream=True,
        ):
            full_response += response.choices[0].delta.get("content", "")
            message_placeholder.markdown(full_response + "▌")
        message_placeholder.markdown(full_response)
    st.session_state.messages.append({"role": "assistant", "content": full_response})
    st.session_state.chat_hist.append({'role':'assistant', 'content':full_response})

# print references:
# add a collapsible section to show reference documents
if len(st.session_state.chat_hist)>0:
    with st.expander("References"):
        st.markdown("Reference documents:")
        for i,doc in enumerate(st.session_state.chat_hist[0]['ref_docs']):
            st.write(f"Reference {i+1}")
            st.write("-"*20)
            st.write(doc)



Overwriting chatbot.py


##Building a local server
The following lines are required to run a streamlit app in google colab. If you run this notebook localy, you only need to open an Anaconda propmt and type:
`streamlit run chatbot.py` while changing your directory (`cd`) to the path of chatbot.py.

In [9]:
!pip install -q streamlit
!npm install localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
added 22 packages from 22 contributors and audited 22 packages in 3.027s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [10]:
!streamlit run chatbot.py &>/content/logs.txt &

This step is also needed because we are running the app on google colab. Please copy the endpoint ip and use it after clicking on the generated url.

In [11]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.141.245.79


Make sure secret_workshop.txt is uploaded to the files section of colab before running the next cell.

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.512s
your url is: https://petite-emus-nail.loca.lt
